In [7]:
import pandas as pd
wd = '/Volumes/PEDS/RI Biostatistics Core/Shared/Shared Projects/Laura/BDC/Projects/Viral Shah/GMI and A1c/'

# Data Cleaning
- HbA1c and mean glucose data were pulled from Kaan's data, CGMND, FLAIR, MOBILE, CITY, DCLP3, WISDM, and data from Peter including participants from DIAMOND, REPLACE, and HypoDE.

In [61]:
# Import Kaan's data (use clean_cgms.py to generate mean glucose file)
# Had to manually edit some names in glucose file to match demographics
kaan = pd.read_csv(wd + 'Data_Clean/kaan_mean_glucose.csv',usecols=['ID','HbA1c','14 Overall Mean'])
kaan.columns = ['ID','HbA1c','Mean Glucose']
# CGMND
cgmnd_a1c = pd.read_csv(wd+'Data_Raw/CGMND-af920dee-2d6e-4436-bc89-7a7b51239837/NonDiabSampleResults.csv')
cgmnd_a1c = cgmnd_a1c[cgmnd_a1c['Analyte'] == "HBA1C"]
cgmnd_a1c = cgmnd_a1c[['PtID','Value']]
cgmnd_a1c.columns = ['ID','HbA1c']
# Calculate mean glucose for each person in CGMND
cgmnd_cgm = pd.read_csv(wd+'Data_Raw/CGMND-af920dee-2d6e-4436-bc89-7a7b51239837/NonDiabDeviceCGM.csv')
ids = cgmnd_cgm['PtID'].unique()
msgs = {'ID':[],'Mean Glucose':[]}
for i in ids:
    d = cgmnd_cgm[(cgmnd_cgm['PtID'] == i) & (cgmnd_cgm['RecordType'] == "CGM")]
    msgs['ID'].append(i)
    msgs['Mean Glucose'].append(d['Value'].mean())
cgmnd_cgm = pd.DataFrame(msgs)
# Merge
cgmnd = pd.merge(cgmnd_a1c,cgmnd_cgm,on='ID')
# FLAIR
flair = pd.read_csv(wd+'Data_Raw/gmiDataFlair 042222 .csv',usecols=['DeidentID','a1cLab','gluMean'])
flair.columns = ['ID','HbA1c','Mean Glucose']
# MOBILE
mobile = pd.read_csv(wd+'Data_Raw/gmiDataMobile 031722 .csv',usecols=['DeidentID','a1cLab','gluMean'])
mobile.columns = ['ID','HbA1c','Mean Glucose']
# DIAMOND, etc
diamond = pd.read_csv(wd+'Data_Raw/gmiData 020222 .csv',usecols=['DeidentID','a1cLab','gluMean'])
diamond.columns = ['ID','HbA1c','Mean Glucose']
# CITY
city = pd.read_csv(wd+'Data_Raw/CITYPublicDataset-344bea7d-8085-4deb-8038-6cb747a744e3/Data Tables/gluIndices RCT.txt',sep='|',usecols=['PtID','gluMean'])